In [1]:
!pip install torch torchvision transformers pillow --quiet



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\Deiaa\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [2]:
import os
print("HF_HOME is set to:", os.environ.get("HF_HOME"))


HF_HOME is set to: D:\huggingface_cache


In [4]:
!pip install open_clip_torch==2.23.0 transformers==4.35.2 pillow


  Using cached open_clip_torch-2.23.0-py3-none-any.whl.metadata (30 kB)
  Using cached transformers-4.35.2-py3-none-any.whl.metadata (123 kB)
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached timm-1.0.15-py3-none-any.whl.metadata (52 kB)
Using cached open_clip_torch-2.23.0-py3-none-any.whl (1.5 MB)
Using cached transformers-4.35.2-py3-none-any.whl (7.9 MB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.2 MB 932.9 kB/s eta 0:00:02
   -------------- ------------------------- 0.8/2.2 MB 1.2 MB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.2 MB 1.3 MB/s eta 0:00:01
   ----------------------- ---------------- 1.3/2.2 MB 1.1 MB/s eta 0:00:01
   ---------------------------- ----------- 1.6/2.2 MB 1.2 MB/s eta 0:00:01
   -----------------------

  You can safely remove it manually.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\Deiaa\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [7]:
!pip install --upgrade transformers


  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached transformers-4.52.4-py3-none-any.whl (10.5 MB)
Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\Deiaa\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [10]:
import os
import torch
from open_clip import create_model_from_pretrained, get_tokenizer

os.environ["HF_HOME"] = r"D:\huggingface_cache"  

repo = "hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224"

model, preprocess = create_model_from_pretrained(repo)
tokenizer = get_tokenizer(repo)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print("Loaded BiomedCLIP via OpenCLIP ✓")


C:\Users\Deiaa\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded BiomedCLIP via OpenCLIP ✓


In [12]:
import os
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Path to your dataset
root_dir = r"D:\LLM\results"

# Define transforms (preprocessing for BioMedCLIP input)
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # BioMedCLIP expects 224x224
    transforms.ToTensor(),
    transforms.Normalize([0.48145466, 0.4578275, 0.40821073], [0.26862954, 0.26130258, 0.27577711]),  # CLIP normalization
])


In [13]:
# Automatically assigns class labels based on subfolder names
dataset = datasets.ImageFolder(root=root_dir, transform=image_transforms)

# To check mapping: e.g., {'glioma': 0, 'meningioma': 1, 'pituitary tumor': 2}
class_to_idx = dataset.class_to_idx
print("Class to Index Mapping:", class_to_idx)


Class to Index Mapping: {'glioma': 0, 'meningioma': 1, 'pituitary tumor': 2}


In [20]:
from torch.utils.data import Dataset
from PIL import Image
import os

class BrainTumorDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        self.classes = sorted(os.listdir(root_dir))  # glioma, meningioma, pituitary tumor
        for label in self.classes:
            class_dir = os.path.join(root_dir, label)
            for file in os.listdir(class_dir):
                if file.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.image_paths.append(os.path.join(class_dir, file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label, img_path


In [21]:
from torch.utils.data import DataLoader

# Use BioMedCLIP's preprocess
dataset = BrainTumorDataset(root_dir=r"D:\LLM\results", transform=preprocess)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)


In [22]:
for image, label, path in dataloader:
    image_path = path[0]
    best_caption, confidence = get_best_caption(image_path, candidate_captions, model, tokenizer, preprocess)
    
    print(f"Image: {os.path.basename(image_path)}")
    print(f"Label: {label[0]}")
    print(f"Caption: {best_caption}")
    print(f"Confidence: {confidence:.4f}")
    print("-" * 50)


TypeError: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'padding_side'

In [14]:
candidate_captions = [
    "MRI showing a glioma tumor",
    "MRI indicating meningioma tumor",
    "MRI with pituitary tumor",
    "Healthy brain MRI",
    "MRI with abnormal tissue growth",
    "MRI with post-surgical signs",
]


In [27]:
def get_best_caption(image_path, captions, model, tokenizer, preprocess):
    image = preprocess(Image.open(image_path).convert("RGB")).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)

        # Tokenize without any extra kwargs — OpenCLIP returns tensors directly
        text_inputs = tokenizer(captions)  # Returns Tensor [len(captions), 77]
        text_inputs = text_inputs.to(device)

        text_features = model.encode_text(text_inputs)

        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        similarity = (image_features @ text_features.T).squeeze(0)
        best_idx = similarity.argmax().item()
        best_caption = captions[best_idx]
        confidence = similarity[best_idx].item()

    return best_caption, confidence


In [28]:
for image, label, path in DataLoader(dataset, batch_size=1):
    image_path = path[0]
    best_caption, confidence = get_best_caption(image_path, candidate_captions, model, tokenizer, preprocess)
    
    print(f"Image: {os.path.basename(image_path)}")
    print(f"Predicted Caption: {best_caption}")
    print(f"Confidence: {confidence:.4f}")
    print("-" * 50)


TypeError: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'padding_side'